# Obtaining the Dataset


In [2]:
import json
import requests
import base64

client id and secret are unique to registered Spotify web app, code is temporary and retrieved from separate Flask app

In [1]:
client_id = '30daab6eacde4b04901c9c6929ed49ea'
client_secret = 'cac7e435e9ca4af39aa9e9bd48beae1d'
code = 'AQA3Mg1Ujlbfa7PXn8DiC4PI0Wossa06dHj4eNsT60pwp01A0U0OjyAfZJW-oFixpyMRUr4oILsws2ozhabJBekZG0RdAMO8UjmVU9nqNHF3Yl-wwpHRNAf0rtG_6oH1gc7IbumDBru9CboDfEtB71gEhElzYmMrieqZ8J3MwrvjbBl-zzL8O_U3U4ZAJ3A_Y2V3K6Xztil3c_Zel2J1ncglm5oz6caGUCRYWc7qWKSrR7VLrkcXisJ68C4J6w'

In [6]:
# # NOTE: THESE ARE FOR USER DATA, NOT GENERAL SEARCH
# auth_header = base64.urlsafe_b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
# token_url = "https://accounts.spotify.com/api/token"
# token_data = {
#     'grant_type':'authorization_code',
#     'code': code,
#     'redirect_uri': 'http://127.0.0.1:5000/' # running the get_code.py program locally
# }
# token_headers = {
#     # "Authorization": f"Basic {encoded_client_creds}"
#     'Authorization': f'Basic {auth_header.decode("ascii")}' # or just encoded_client_creds here
# }
# r = requests.post(token_url, data=token_data, headers=token_headers)
# print(r.json()) # not needed until retrieving from user data
# token_response_data = r.json()

## Retrieve songs and lyrics

In [8]:
auth_header = base64.urlsafe_b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
token_url = "https://accounts.spotify.com/api/token"
token_data = {
    'grant_type':'client_credentials',
    'code': code,
    'redirect_uri': 'http://127.0.0.1:5000/' # running the get_code.py program locally
}
token_headers = {
    # "Authorization": f"Basic {encoded_client_creds}"
    'Authorization': f'Basic {auth_header.decode("ascii")}' # or just encoded_client_creds here
}

In [9]:
r = requests.post(token_url, data=token_data, headers=token_headers)
print(r.json()) # note: diff from user api functions; only searches publically available data
token_response_data = r.json()

{'access_token': 'BQCoULPu03IFQU_yuqa1fZO0D3xvadDEX29gzZ3oZhvf5AHnqmBne8tEif1NYrhgIIesuvrolXgGNWnVqp7LV4vOMp_rAvcmFlVjNzXWWqERGI1-BkM', 'token_type': 'Bearer', 'expires_in': 3600}


In [25]:
print(r.json()['access_token'])

BQCoULPu03IFQU_yuqa1fZO0D3xvadDEX29gzZ3oZhvf5AHnqmBne8tEif1NYrhgIIesuvrolXgGNWnVqp7LV4vOMp_rAvcmFlVjNzXWWqERGI1-BkM


In [26]:
# Green Day, The Strokes, The Velvet Underground, Slaughter Beach, Dog, Fleetwood Mac, Mitski, and boygenius
# 0          1            2                       3                     4              5           6

# got these manually from artist's IDs in their share links
artist_ids = ['7oPftvlwr6VrsViSDV7fJY', '0epOFNiUfyON9EYx7Tpr6V', '1nJvji2KIlWSseXRSlNYsC', '3lWVgSwutPsiJ8Awm7OTKU', '08GQAI4eElDnROBrJRGE0X', '2uYWxilOVlUdk4oV9DvwqK', '1hLiboQ98IQWhpKeP9vRFw']
headers = {
    'Authorization': f"Bearer {token_response_data['access_token']}",
    'Content-Type': 'application/json'
}

In [27]:
# loop through artists to retrieve albums
responses=[]
for id in artist_ids:
  responses.append(requests.get(f"https://api.spotify.com/v1/artists/{id}/albums", headers=headers).json())

In [28]:
responses

[{'href': 'https://api.spotify.com/v1/artists/7oPftvlwr6VrsViSDV7fJY/albums?include_groups=album,single,compilation,appears_on&offset=0&limit=20',
  'items': [{'album_group': 'album',
    'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7oPftvlwr6VrsViSDV7fJY'},
      'href': 'https://api.spotify.com/v1/artists/7oPftvlwr6VrsViSDV7fJY',
      'id': '7oPftvlwr6VrsViSDV7fJY',
      'name': 'Green Day',
      'type': 'artist',
      'uri': 'spotify:artist:7oPftvlwr6VrsViSDV7fJY'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
   

In [30]:
responses[0]['items'][0]['id']

'47l8TkJCLzHaekHImz1e9g'

Each entry in `responses` is a response containing all of artist `i`'s albums. Loop through `responses` and replace the response by a list of album Spotify IDs

In [33]:
album_ids = []
for i in range(len(responses)):
  album_list = []
  print("len(responses[i]['items']):", len(responses[i]['items']))
  for j in range(len(responses[i]['items'])):
    album_list.append(responses[i]['items'][j]['id'])
  album_ids.append(album_list)

len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 20
len(responses[i]['items']): 16
len(responses[i]['items']): 20


In [32]:
album_ids

[['47l8TkJCLzHaekHImz1e9g',
  '4AFF6Md43PaWKv5lSMAysd',
  '4R802IhlcVX7z3m7xyp7zC',
  '7igexfyHGnY4qMAyOzc4q8',
  '2nrqcI2e61NN375lBaQIg0',
  '6pjuzPdAkJPdNBiKzcloXL',
  '7ij8wQxCAexZiXZbMOHcxE',
  '5a3LqvNt2nv1B4aRKXmgOV',
  '7prfPBMbgDyhq7kdpM0R9v',
  '1t9rzuEEFVHPdqRYzsaWC8',
  '71rrqHKVkUFx2CSVfxrrLs',
  '6AFJIZmd1zWQDBBwHyeo9w',
  '1AHZd3C3S8m8fFrhFxyk79',
  '5Qhn2FpGWmTjCuntF09j7g',
  '3uxqKqchyNKPHdjzMrlmOC',
  '3ifIxGNsG1XmLdoanRRIWB',
  '3x2uer6Xh0d5rF8toWpRDA',
  '7d3nOmFvL51roNElAdpi9d',
  '4uG8q3GPuWHQlRbswMIRS6',
  '1UShup0VvfxhxS7j3Omxh2'],
 ['2j5jqlI0NvqlXAy9C740pZ',
  '2xkZV2Hl1Omi8rk2D7t5lN',
  '4WnkQO4xD9ljQooB3VIxCV',
  '6Jx4cGhWHewTcfKDJKguBQ',
  '1HQ61my1h3VWp2EBWKlp0n',
  '3HFbH1loOUbqCyPsLuHLLh',
  '2k8KgmDp9oHrmu0MIj4XDE',
  '1SQjs5LxCj7J5WIZYg3h1D',
  '6sKy9KhjloDKHcoyfY8UgZ',
  '5LC9CjBeKgGdE52txji0kg',
  '7jbQIpfQZTVBn8J7FzMyaF',
  '1JS6yTcVopsE6ROReK30ik',
  '0oq8O4Hk2Gl6kMYBHRN4Y9',
  '19jQHcoF9tnrrKyB7CbWU0',
  '7Ll34Ibt4hcqonus8MR6o3',
  '5mRPvdITWYckQAX3

Retrieve all track IDs for each album

In [34]:
track_ids = []
for artist_discog in album_ids: # there are 7 artist_discogs, each being a list
  tracks = []
  for album_id in artist_discog:
    tracks.append(requests.get(f"https://api.spotify.com/v1/albums/{album_id}/tracks", headers=headers).json())
  track_ids.append(tracks)

In [46]:
# 3st artist tracks, 1st album, 1st song id
track_ids[3][0]['items'][0]['id']

'5h8twYp6D3Tus5XzNTjcvX'

Compile all track ID's into a single list for each artist

retrieve lyrics

In [45]:
lyricurl = 'https://spotify-lyric-api-984e7b4face0.herokuapp.com/?trackid=5h8twYp6D3Tus5XzNTjcvX'
lyric_data = {
    "trackid": '5h8twYp6D3Tus5XzNTjcvX'
}
words = requests.get(lyricurl, data=lyric_data)
words.json()

{'error': False,
 'syncType': 'LINE_SYNCED',
 'lines': [{'startTimeMs': '34930',
   'words': 'Not a poet, not even close',
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '39020',
   'words': "Surfin' New Jersey",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '42840',
   'words': "The one I know isn't here",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '47060',
   'words': 'They look at me funny, hee-hee-hee',
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '53390', 'words': '♪', 'syllables': [], 'endTimeMs': '0'},
  {'startTimeMs': '84470',
   'words': 'Get in with our suits zipped up',
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '88490',
   'words': 'Bobbing a while',
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '92480',
   'words': "I try, but I don't catch much",
   'syllables': [],
   'endTimeMs': '0'},
  {'startTimeMs': '96020',
   'words': 'Driving home on the turnpike',
   'syllables': [],
   'e